In [ ]:
# Proyecto de desarrollo de sistema de IA basado en alquileres de pisos – Segunda fase
# Carmen De Los Ángeles Camacho Tejada - 02/03/2025
# Ciencia De Datos - UNIR
# Profesor Alan Sastre - Módulo 2

# Importamos las librerias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


*_CARGA Y EXPLORACIÓN INICIAL DEL DATASET_*

In [ ]:
df = pd.read_csv("madrid_dataset_idealista.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

Para la siguiente parte de la exploración del dataset, dejo un pequeño esquema con los pasos que voy a seguir a continuación:
- Elimina duplicados con drop_duplicates().
- Revisa valores nulos con isnull().sum().
- Muestra los tipos de datos actuales con dtypes.
- Convierte fechas (last update) a formato datetime, evitando errores con errors='coerce'.
- Convierte variables categóricas (district, subdistrict, orientation) a category.
- Convierte variables binarias (garage_included, furnished, balcony) a int.
- Muestra estadísticas de price, floor_area y year_built para detectar valores extraños.
- Identifica valores fuera de rango, por ejemplo, negativos en price o floor_area.


In [ ]:
# Eliminar duplicados
df = df.drop_duplicates()

In [ ]:
# Revisar valores nulos
df.isnull().sum()

In [ ]:
# Identificar inconsistencias en los tipos de datos
df.dtypes

In [ ]:
# Convertir la columna de fecha a formato datetime
if 'last update' in df.columns:
    df['last update'] = pd.to_datetime(df['last update'], errors='coerce')

In [ ]:
# Convertir variables categóricas a tipo category
categorical_columns = ['district', 'subdistrict', 'orientation']
for col in categorical_columns:
    if col in df.columns:
        df[col] = df[col].astype('category')

In [ ]:
# Convertir variables binarias a 0/1
binary_columns = ['garage_included', 'furnished', 'balcony']
for col in binary_columns:
    if col in df.columns:
        df[col] = df[col].astype(int)

In [ ]:
# Verificar valores extraños en price, floor_area y year_built
df[['price', 'floor_area', 'year_built']].describe()

In [ ]:
# Verificar si hay valores negativos o fuera de un rango lógico
for col in ['price', 'floor_area', 'year_built']:
    if col in df.columns:
        print(f"\nValores fuera de rango en {col}:")
        print(df[df[col] < 0])

*_Limpieza y validación de datos_*


Para el siguiente paso, dejo a continuación un pequeño esquema de aquello que voy a realizar:

- Manejo de valores nulos
- Variables categóricas: Se rellenan con la moda
- Variables numéricas: Se prueban SimpleImputer (mediana), KNNImputer e IterativeImputer.
- Se comparan las estadísticas de cada método para elegir el mejor.
- Detección y tratamiento de outliers
- Boxplots para visualizar valores extremos.
- IQR para detectar outliers en price, floor_area, bedrooms.
- Winsorization en lugar de eliminar datos para evitar pérdida excesiva.
- Transformación de variables con sesgo alto
- Se usa log-transform (np.log1p()).
- Se usa Box-Cox / Yeo-Johnson para mejorar normalidad.
- Se comparan histogramas antes y después de la transformación.
- Se revisa el sesgo (skew()) para verificar la mejora.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from scipy import stats
from sklearn.preprocessing import PowerTransformer

In [ ]:
### 1. Manejo de valores nulos

# Variables categóricas: rellenar con la moda
cat_cols = ['district', 'subdistrict', 'orientation']
for col in cat_cols:
    if col in df.columns:
        mode_value = df[col].mode()[0]  # Moda
        df.fillna({col: mode_value}, inplace=True)  # Solución recomendada

In [ ]:
# Variables numéricas: aplicar diferentes imputers
num_cols = ['price', 'floor_area', 'year_built', 'bedrooms', 'bathrooms']

# SimpleImputer con la mediana
simple_imputer = SimpleImputer(strategy='median')
df_simple = df.copy()
df_simple[num_cols] = simple_imputer.fit_transform(df_simple[num_cols])

# KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
df_knn = df.copy()
df_knn[num_cols] = knn_imputer.fit_transform(df_knn[num_cols])

# IterativeImputer
iter_imputer = IterativeImputer(max_iter=10, random_state=42)
df_iter = df.copy()
df_iter[num_cols] = iter_imputer.fit_transform(df_iter[num_cols])

# Evaluar cuál imputación es mejor (comparando estadísticas)
print("SimpleImputer Stats:\n", df_simple[num_cols].describe())
print("\nKNNImputer Stats:\n", df_knn[num_cols].describe())
print("\nIterativeImputer Stats:\n", df_iter[num_cols].describe())

Comparación de Imputers:

1. SimpleImputer (Mediana o Moda)
Ventajas: Método sencillo y rápido. No altera significativamente la distribución original de los datos. Útil si hay pocos valores nulos.
Desventajas: Puede no capturar relaciones entre variables. Mantiene valores repetidos (ejemplo: floor_area = 75 en 25%-75%).
Aquí, floor_area tiene una alta concentración en 75, lo que sugiere que la mediana no representa bien la distribución real. No introduce valores fuera del rango original.
2. KNNImputer
Ventajas: Considera valores cercanos en otras variables para imputar datos. Puede capturar patrones en los datos mejor que el SimpleImputer.
Desventajas: Más costoso computacionalmente. Puede distorsionar datos si la cantidad de nulos es alta.
Aquí, floor_area tiene una media más alta (100.42 vs. 83.26 en SimpleImputer), lo que sugiere que KNN está elevando los valores imputados. year_built muestra una mayor dispersión, lo cual puede ser bueno o malo dependiendo de la estructura real de los datos.
3. IterativeImputer
Ventajas: Usa regresión para estimar valores, lo que permite una imputación más precisa. Aprende de todas las variables disponibles.
Desventajas: Computacionalmente más costoso. Puede generar valores fuera del rango original.
Aquí, floor_area tiene valores negativos (-29.92), lo cual es inválido. year_built tiene valores fuera del rango normal (2109), lo que sugiere que está extrapolando más de lo debido.

Para esta actividad, no se especifica nada sobre el costo computacional, por ende, se usará KNNImputer, ya que, captura patrones complejos y tienes pocos valores nulos.

In [ ]:
# KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
df[num_cols] = knn_imputer.fit_transform(df[num_cols])

In [ ]:
# Para los gráficos, se busca aplicar un estilo visual profesional
sns.set_style("darkgrid")
plt.rcParams.update({
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "figure.figsize": (12, 6)
})

### 2. Detección y Tratamiento de Outliers

# Visualización con Boxplots antes del tratamiento
plt.figure(figsize=(12, 6))
sns.boxplot(data=df[['price', 'floor_area', 'bedrooms']], palette="viridis")
plt.xticks(rotation=45)
plt.title("Boxplot de Variables Numéricas Antes de Tratar Outliers", fontsize=14)
plt.show()


In [ ]:
# Función para tratar outliers con IQR (Winsorization)
def treat_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    return df


In [ ]:
# Aplicar Winsorization a price, floor_area y bedrooms
for col in ['price', 'floor_area', 'bedrooms']:
    df = treat_outliers_iqr(df, col)

In [ ]:
# Visualización con Boxplots después del tratamiento
plt.figure(figsize=(12, 6))
sns.boxplot(data=df[['price', 'floor_area', 'bedrooms']], palette="deep")
plt.xticks(rotation=45)
plt.title("Boxplot de Variables Numéricas Después de Tratar Outliers", fontsize=14)
plt.show()

In [ ]:
### 3. Transformación de Datos con Sesgo Alto

# Verificar sesgo antes de la transformación
print("\n Skewness antes de la transformación:")
print(df[['price', 'floor_area']].skew())

# Aplicar Transformaciones
df['price_log'] = np.log1p(df['price'])
df['floor_area_log'] = np.log1p(df['floor_area'])

# Aplicar PowerTransformer (Box-Cox/Yeo-Johnson)
pt = PowerTransformer(method='yeo-johnson')
df[['price_bc', 'floor_area_bc']] = pt.fit_transform(df[['price', 'floor_area']])


In [ ]:
# Comparación de Histogramas antes y después de la transformación
fig, axes = plt.subplots(1, 3, figsize=(18, 5), constrained_layout=True)
colors = ["#7babb3", "#ff7f98", "#8ac767"]  # Azul, Rojo, Verde

sns.histplot(df['price'], bins=50, kde=True, color=colors[0], ax=axes[0])
axes[0].set_title(" Distribución Original de Price", fontsize=12)
axes[0].axvline(df['price'].mean(), color='black', linestyle='dashed', linewidth=1)

sns.histplot(df['price_log'], bins=50, kde=True, color=colors[1], ax=axes[1])
axes[1].set_title(" Distribución con Log Transform", fontsize=12)
axes[1].axvline(df['price_log'].mean(), color='black', linestyle='dashed', linewidth=1)

sns.histplot(df['price_bc'], bins=50, kde=True, color=colors[2], ax=axes[2])
axes[2].set_title(" Distribución con Box-Cox/Yeo-Johnson", fontsize=12)
axes[2].axvline(df['price_bc'].mean(), color='black', linestyle='dashed', linewidth=1)

plt.show()


In [ ]:
# Verificar sesgo después de la transformación
print("\n Skewness después de la transformación:")
print(df[['price_log', 'floor_area_log', 'price_bc', 'floor_area_bc']].skew())

*_Análisis exploratorio de datos (EDA)_*

Para el siguiente paso, dejo un esquema de lo que voy a hacer a continuación:

- Análisis univariante:
    - Histogramas para variables numéricas.
    - Diagramas de densidad para estudiar distribuciones.
    - Estadísticas descriptivas
- Análisis bivariante:
    - Correlación entre variables numéricas.
    - Gráficos de dispersión para ver relación entre price y floor_area.
    - Gráficos categóricos para ver distribución de price según district.
- Análisis multivariante:
    - Gráficos de pares para identificar relaciones entre múltiples variables.
    - Mapas de calor de correlación para detectar multicolinealidad.
    - Aplicar PCA para explorar la dimensionalidad de los datos.


In [ ]:
# Histograma para 'price'
plt.figure(figsize=(10,6))
sns.histplot(df['price'], kde=True, color="skyblue", bins=30, edgecolor="black")
plt.title('Distribución de Precio', fontsize=16)
plt.xlabel('Precio (€)', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.grid(True)
plt.show()

# Histograma para 'floor_area'
plt.figure(figsize=(10,6))
sns.histplot(df['floor_area'], kde=True, color="coral", bins=30, edgecolor="black")
plt.title('Distribución de Área de Piso', fontsize=16)
plt.xlabel('Área del Piso (m²)', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
# Diagrama de Densidad para 'price'
plt.figure(figsize=(10,6))
sns.kdeplot(df['price'], fill=True, color="purple", lw=2)
plt.title('Distribución de Densidad del Precio', fontsize=16)
plt.xlabel('Precio (€)', fontsize=12)
plt.ylabel('Densidad', fontsize=12)
plt.grid(True)
plt.show()

# Diagrama de Densidad para 'floor_area'
plt.figure(figsize=(10,6))
sns.kdeplot(df['floor_area'], fill=True, color="chocolate", lw=2)
plt.title('Distribución de Densidad del Área del Piso', fontsize=16)
plt.xlabel('Área del Piso (m²)', fontsize=12)
plt.ylabel('Densidad', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
# Estadísticas Descriptivas
df.describe(percentiles=[0.25, 0.5, 0.75])

In [ ]:
# Eliminar las columnas no numéricas
df_numeric = df.select_dtypes(include=[np.number])

# Calcular la matriz de correlación
corr_matrix = df_numeric.corr()

# Mostrar el mapa de calor de la correlación
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5, vmin=-1, vmax=1)
plt.title('Mapa de Calor de Correlación', fontsize=16)
plt.show()




In [ ]:
# Gráfico de Dispersión entre 'price' y 'floor_area'
plt.figure(figsize=(10,6))
sns.scatterplot(x='floor_area', y='price', data=df, color='teal', alpha=0.7)
plt.title('Relación entre Área del Piso y Precio', fontsize=16)
plt.xlabel('Área del Piso (m²)', fontsize=12)
plt.ylabel('Precio (€)', fontsize=12)
plt.grid(True)
plt.show()


In [ ]:
# Gráfico de Caja para 'price' según 'district'
plt.figure(figsize=(12,6))
sns.boxplot(x='district', y='price', data=df, hue='district', palette='Set2', showfliers=False)
plt.title('Distribución del Precio por Distrito', fontsize=16)
plt.xlabel('Distrito', fontsize=12)
plt.ylabel('Precio (€)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(True)
plt.show()


In [ ]:
# Gráfico de Violín para 'price' según 'district'
plt.figure(figsize=(12,6))
sns.violinplot(x='district', y='price', data=df, hue='district', split=True, palette='muted', inner="quart")
plt.title('Distribución del Precio por Distrito', fontsize=16)
plt.xlabel('Distrito', fontsize=12)
plt.ylabel('Precio (€)', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(True)
plt.show()


In [ ]:
# Gráfico de Dispersión entre 'price' y 'year_built'
plt.figure(figsize=(10,6))
sns.scatterplot(x='year_built', y='price', data=df, color='indianred', alpha=0.7)
plt.title('Relación entre Año de Construcción y Precio', fontsize=16)
plt.xlabel('Año de Construcción', fontsize=12)
plt.ylabel('Precio (€)', fontsize=12)
plt.grid(True)
plt.show()


In [ ]:
# Gráfico de Regresión para 'price' y 'floor_area'
plt.figure(figsize=(10,6))
sns.regplot(x='floor_area', y='price', data=df, scatter_kws={'s': 30}, line_kws={'color': 'red'})
plt.title('Regresión entre Área del Piso y Precio', fontsize=16)
plt.xlabel('Área del Piso (m²)', fontsize=12)
plt.ylabel('Precio (€)', fontsize=12)
plt.grid(True)
plt.show()


In [ ]:
# Gráfico de barras para 'price' según 'terrace'
plt.figure(figsize=(10,6))
sns.barplot(x='terrace', y='price', data=df, palette='pastel', hue= 'terrace', legend=False)
plt.title('Precio según la Presencia de Terraza', fontsize=16)
plt.xlabel('Tiene Terraza', fontsize=12)
plt.ylabel('Precio (€)', fontsize=12)
plt.grid(True)
plt.show()


In [ ]:
import plotly.express as px

fig = px.scatter(df, x='floor_area', y='price', title='Relación entre Área y Precio',
                 labels={'floor_area': 'Área (m²)', 'price': 'Precio'},
                 color='district', # Si quieres diferenciar por distrito
                 template='plotly_dark')  # O 'ggplot2', 'seaborn', etc.
fig.show()


In [ ]:
fig = px.box(df, x='district', y='price', title='Distribución de Precio según Distrito',
             labels={'district': 'Distrito', 'price': 'Precio'},
             template='plotly_dark')
fig.show()


In [ ]:
import plotly.graph_objects as go

corr_matrix = df_numeric.corr()
fig = go.Figure(data=go.Heatmap(z=corr_matrix.values,
                               x=corr_matrix.columns,
                               y=corr_matrix.columns,
                               colorscale='Viridis'))
fig.update_layout(title='Mapa de Calor de Correlación')
fig.show()


In [ ]:
fig = px.scatter_matrix(df, dimensions=['floor_area', 'price', 'bedrooms', 'bathrooms'],
                        title='Gráfico de Pares', color='district')
fig.show()


In [ ]:
fig = px.scatter_3d(df, x='floor_area', y='price', z='bedrooms', color='district',
                    title='Gráfico de Dispersión 3D: Área, Precio y Habitaciones',
                    labels={'floor_area': 'Área (m²)', 'price': 'Precio', 'bedrooms': 'Habitaciones'})
fig.show()


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Seleccionamos las columnas numéricas
df_numeric = df[['floor_area', 'price', 'bedrooms', 'bathrooms']]

# Normalizamos las variables
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_numeric)

# Realizamos PCA
pca = PCA(n_components=2)
pca_components = pca.fit_transform(df_scaled)

# Creamos un DataFrame con las dos primeras componentes principales
pca_df = pd.DataFrame(data=pca_components, columns=['PCA1', 'PCA2'])

# Agregamos el 'district' para la visualización
pca_df['district'] = df['district']

# Graficamos los componentes principales
fig = px.scatter(pca_df, x='PCA1', y='PCA2', color='district', title='PCA: Componentes Principales',
                 labels={'PCA1': 'Componente Principal 1', 'PCA2': 'Componente Principal 2'})
fig.show()


In [ ]:
# Mostrar la varianza explicada por cada componente
explained_variance = pca.explained_variance_ratio_
print(f"Varianza explicada por PCA1: {explained_variance[0]:.2f}")
print(f"Varianza explicada por PCA2: {explained_variance[1]:.2f}")


*_Preparación de los datos para el modelado_*

Aquí le adjunto un listado de los pasos que voy a seguir:

- Codificación de Variables Categóricas: One-Hot Encoding para variables con múltiples categorías.
- Escalado de Variables Numéricas: Probamos StandardScaler.
- Segmentación en Entrenamiento y Validación: Usamos train_test_split para dividir los datos.
- Pipeline de Preprocesamiento: Combinamos todos los pasos anteriores en un Pipeline para facilitar el flujo de trabajo.


In [ ]:
# Lista de columnas que no son necesarias para las predicciones:
# web_id: Aunque es único, no es útil para la predicción del precio.
# url: Solo es un identificador de la propiedad, no aporta información útil para el modelo.
# title: Puede ser útil en un análisis de texto, pero en general, no es necesario para este ejercicio.
# professional_name: Es un dato sobre la agencia, pero puede no tener un gran impacto directo en el precio.
# last update: Es una variable temporal que, en la mayoría de los casos, no tiene relevancia para el precio del alquiler.
columns_to_drop = [
    'web_id', 'url', 'title', 'professional_name', 'last_update'
]
# Eliminar las columnas
df.drop(columns=columns_to_drop, axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

# Seleccionamos las columnas numéricas para escalar
categorical_cols = ['type', 'floor', 'orientation', 'district', 'subdistrict', 'location']

# Usar LabelEncoder para convertir las columnas categóricas a valores numéricos
le = LabelEncoder()
for col in categorical_cols:
    df[col] = le.fit_transform(df[col])

binary_cols = [
    'private_owner', 'second_hand', 'lift', 'equipped_kitchen', 'fitted_wardrobes', 'air_conditioning',
    'terrace', 'storeroom', 'swimming_pool', 'garden_area'
]
for col in binary_cols:
    df[col] = df[col].astype(int)

numerical_cols = [
    'deposit', 'floor_built', 'floor_area', 'year_built', 'bedrooms', 'bathrooms',
    'garage_included', 'furnished', 'balcony', 'postalcode', 'floor_area_log', 'floor_area_bc'
]


In [ ]:
# Pipeline de preprocesamiento para las variables categóricas
# Obtener todas las categorías posibles combinando X_train y X_val
categorias_completas = [np.unique(df[col]) for col in categorical_cols]

# OneHotEncoder con categorías predefinidas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(categories=categorias_completas, drop='first', handle_unknown='ignore'))
])

# Pipeline de preprocesamiento para variables binarias
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
])

# Pipeline de preprocesamiento para las variables numéricas
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [ ]:
# Crear el preprocesador final
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
        ('bin', binary_transformer, binary_cols)
    ], remainder='passthrough'
)

Primero, se hará el Modelado con Regresión para predecir el precio.

In [ ]:
# Dividir el conjunto de datos en entrenamiento y validación
X = df.drop(columns=['price'])  # Variables independientes (sin la columna 'price')
y = df['price']  # Variable dependiente (target)

# Dividir en entrenamiento y validación (80% / 20%)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

*_Modelado_*
- _Regresión (Predicción del Precio)_
+ Modelos a probar:
- Regresión Lineal
- Árboles de Decisión
- Random Forest
- GradientBoostingRegressor
- SVR
- KNeighborsRegressor
- Red Neuronal Pequeña con TensorFlow

+ Métricas de evaluación:
- MSE (Error Cuadrático Medio)
- RMSE (Raíz del MSE)
- MAPE (Error Absoluto Porcentual Medio)
- Validación Cruzada



Explicación de los modelos a usar y de las métricas con las que se evaluará:
- Regresión Lineal: Es un modelo simple, ideal como referencia.
- Árboles de Decisión: Útil para capturar no linealidades y relaciones complejas en los datos.
- Random Forest: Un modelo de ensamblaje que mejora la robustez y la precisión de los árboles de decisión.
- SVR (Support Vector Regression): Bueno para capturar relaciones no lineales.
- K-Neighbors Regressor: Utiliza la proximidad entre las muestras para predecir los valores.
- Gradient Boosting: Un modelo de ensamblaje basado en árboles que es muy efectivo en problemas de regresión.
- Red Neuronal: Implementada en TensorFlow, modela relaciones complejas y no lineales en los datos.
- MSE (Mean Squared Error): Penaliza fuertemente los errores grandes.
- RMSE (Root Mean Squared Error): Raíz cuadrada del MSE, más fácil de interpretar en las mismas unidades que el objetivo (precio).
- MAPE (Mean Absolute Percentage Error): Mide el error porcentual medio entre las predicciones y los valores reales.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from keras.src.layers import Dense
from keras import Input, Sequential


In [ ]:
# Crear un pipeline de modelado con el preprocesador y el modelo
def create_model(model):
    return Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])


In [ ]:
# Modelos a probar
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'SVR': SVR(),
    'K-Neighbors': KNeighborsRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'Neural Network': None  # Se verá por separado, al ser con TensorFlow
}

# Diccionario de hiperparámetros solo para los modelos que admiten GridSearch
param_grids = {
    'Decision Tree': {
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 4]
    },
    'Random Forest': {
        'model__n_estimators': [50, 100],
        'model__max_depth': [5, 10, 20],
        'model__min_samples_split': [2, 5],
        'model__min_samples_leaf': [1, 2]
    },
    'SVR': {
        'model__kernel': ['linear', 'rbf'],
        'model__C': [0.1, 1, 10]
    },
    'K-Neighbors': {
        'model__n_neighbors': [3, 5, 10],
        'model__weights': ['uniform', 'distance']
    },
    'Gradient Boosting': {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [3, 5, 10]
    }
}

In [ ]:
# Esta celda dura casi 10 minutos al hacer muchas cosas de golpe
# Resultados para almacenar métricas de cada modelo
results = {}

# Entrenamiento y evaluación de los modelos de Scikit-Learn
for model_name, model in models.items():
    if model_name != 'Neural Network':  # Modelos de Scikit-Learn
        pipeline = create_model(model)

        if model_name in param_grids:  # Si el modelo tiene hiperparámetros ajustables
            print(f"\nEjecutando GridSearch para {model_name}...")
            grid_search = GridSearchCV(pipeline, param_grids[model_name],
                                       cv=5, scoring='neg_mean_squared_error',
                                       n_jobs=-1, verbose=2)
            grid_search.fit(X_train, y_train)

            # Guardar el mejor modelo encontrado
            best_model = grid_search.best_estimator_
            print(f"Mejores parámetros para {model_name}: {grid_search.best_params_}")
        else:
            best_model = pipeline.fit(X_train, y_train)

        # Predicciones
        y_pred = best_model.predict(X_val)

        # Métricas
        mse = mean_squared_error(y_val, y_pred)
        rmse = root_mean_squared_error(y_val, y_pred)  # Calculamos RMSE con la nueva función
        mape = mean_absolute_percentage_error(y_val, y_pred)

        # Guardamos los resultados
        results[model_name] = {'MSE': mse, 'RMSE': rmse, 'MAPE': mape}
    else:  # Para la red neuronal con TensorFlow
        # Preprocesar los datos
        X_train_nn = preprocessor.fit_transform(X_train)
        X_val_nn = preprocessor.transform(X_val)

        # Crear una red neuronal sencilla
        model_nn = Sequential([
            Input(shape=(X_train_nn.shape[1],)),
            Dense(128, activation='relu'),
            Dense(64, activation='relu'),
            Dense(1)  # Una única salida (el precio)
])

        model_nn.compile(optimizer='adam', loss='mean_squared_error')


        # Entrenamiento de la red neuronal
        model_nn.fit(X_train_nn, y_train, epochs=50, batch_size=32, verbose=0)

        # Predicciones
        y_pred_nn = model_nn.predict(X_val_nn)

        # Métricas
        mse_nn = mean_squared_error(y_val, y_pred_nn)
        rmse_nn = root_mean_squared_error(y_val, y_pred_nn)  # Calculamos RMSE con la nueva función
        mape_nn = mean_absolute_percentage_error(y_val, y_pred_nn)

        # Guardamos los resultados
        results['Neural Network'] = {'MSE': mse_nn, 'RMSE': rmse_nn, 'MAPE': mape_nn}



In [ ]:
# Evaluación de modelos con validación cruzada
for model_name, model in models.items():
    if model_name != 'Neural Network':  # Los modelos de sklearn
        pipeline = create_model(model)
        cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
        mean_cv_score = -cv_scores.mean()  # Convertimos de negativo a positivo
        print(f"{model_name} - MSE (Cross-Validation): {mean_cv_score}")


In [ ]:
# Mostrar los resultados
results_df = pd.DataFrame(results)
results_df

Según la validación cruzada:
1. El MSE de la regresión lineal es relativamente alto, lo que indica que el modelo no está ajustando bien los datos. Puede que no esté capturando bien las relaciones no lineales presentes en el conjunto de datos.
2. El modelo de Árbol de Decisión tiene un MSE bajo, lo que sugiere que es capaz de ajustar bien el modelo a los datos. Sin embargo, los árboles de decisión son propensos al sobreajuste, por lo que se debe verificar su rendimiento en datos no vistos y considerar regularización si es necesario.
3. El modelo de Random Forest tiene el MSE más bajo de todos, lo que indica que está haciendo un excelente trabajo ajustando el modelo a los datos. Random Forest generalmente ofrece un buen balance entre bajo error y resistencia al sobreajuste.
4. SVR (Support Vector Regression): El modelo SVR tiene un MSE extremadamente alto, lo que indica que está ajustando mal los datos. Esto podría ser debido a un mal ajuste de los hiperparámetros, como el kernel, la regularización, etc. Es probable que se necesite una mayor sintonización o que no sea adecuado para este problema específico.
5. El modelo de K-Neighbors tiene un MSE relativamente alto. Aunque K-Neighbors puede ser útil en problemas donde las relaciones entre las características son locales, su rendimiento puede verse afectado por la elección del número de vecinos.
6. El Gradient Boosting muestra un MSE moderado, lo que indica que está ajustando bien los datos, pero no tanto como Random Forest. Gradient Boosting es un modelo muy potente, pero puede ser sensible a la sobreajuste si no se ajustan correctamente los hiperparámetros.


En conclusión, Random Forest es el modelo más prometedor.

Aquí está la evaluación de los resultados basados en las métricas MSE, RMSE y MAPE para los modelos:

1. La regresión lineal tiene un MSE y RMSE relativamente altos en comparación con otros modelos. Sin embargo, su MAPE es bajo, lo que indica que, a pesar de que el error cuadrático es mayor, el porcentaje de error relativo sobre los valores reales no es tan grande.
2. El árbol de decisión tiene un desempeño excelente con valores extremadamente bajos para MSE, RMSE y MAPE, lo que sugiere que está ajustando muy bien a los datos. Sin embargo, es importante revisar si este modelo está sobre ajustando los datos, ya que los árboles de decisión pueden ser propensos a ello.
3. Random Forest tiene el mejor desempeño en todas las métricas. El MSE y RMSE son los más bajos, lo que indica un excelente ajuste, y el MAPE también es extremadamente bajo, lo que refleja que los errores relativos son mínimos. Este modelo es el más robusto y preciso en este conjunto de resultados.
4. El modelo SVR tiene un MSE y RMSE muy altos, lo que indica un pobre desempeño. Además, el MAPE es relativamente alto, lo que sugiere que los errores del modelo son grandes en relación con los valores reales. Esto sugiere que el modelo no está ajustando bien los datos, y podría ser necesario ajustar los parámetros (como el kernel, la regularización, etc.) o probar otro enfoque.
5. K-Neighbors también tiene un MSE y RMSE altos, lo que indica que no está ajustando bien a los datos. Además, el MAPE es alto, lo que muestra que hay un gran error relativo. Al igual que SVR, podría necesitar ajustes en los hiperparámetros.
6. Gradient Boosting tiene un rendimiento bastante bueno con un MSE y RMSE bajos. Aunque no es tan bueno como Random Forest, sus valores son aún excelentes, y su MAPE también es bajo, lo que sugiere que el modelo está funcionando bien en términos de error relativo.
7. La red neuronal tiene un MSE y RMSE más altos que Random Forest y Gradient Boosting, pero su MAPE es aceptable. Es probable que la red neuronal esté haciendo un buen trabajo ajustando los datos, pero necesitaría ser afinada más en términos de arquitectura y parámetros de entrenamiento.


Viendo todo lo dicho, Random Forest es el modelo más recomendable para este problema, dado su desempeño general superior.
En el caso de querer experimentar con otros modelos, Decision Tree y Gradient Boosting también tienen buenos resultados y podrían ser opciones viables.
SVR y K-Neighbors deben ser ajustados o descartados, ya que no están funcionando bien con este conjunto de datos.